In [1]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# For topic modeling
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd
# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amaknabil2001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amaknabil2001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amaknabil2001\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
documents = [
    "Rafael Nadal Joins Roger Federer in Missing U.S Open",
    "Rafael Nadal Is out of the australian open",
    "Biden Announces Virus Measures",
    "Biden's Virus plans meet reality",
    "where biden's virus plan stands"
]

In [3]:
# Create a set of English stopwords
stop_words = set(stopwords.words('english'))

# Initialize a WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize the text into words and convert to lowercase
    tokens = word_tokenize(text.lower())
    
    # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token.isalnum()]
    
    # Remove stopwords from the tokens
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize each token
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Return the preprocessed tokens
    return tokens

# Apply preprocessing to a list of documents
preprocessed_documents = [preprocess_text(doc) for doc in documents]


In [12]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents) 
# Filter out tokens that appear in less than 15 documents or more than 50% of the documents

# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(3, 1), (4, 1), (5, 1), (7, 1)], [(8, 1), (9, 1), (10, 1), (11, 1)], [(9, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(9, 1), (11, 1), (13, 1), (15, 1)]]


In [15]:
# Run LDA
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15) 

In [9]:
preprocessed_documents

[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafael', 'nadal', 'australian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

In [16]:
# empty list to store dominant topic labels for each document
article_labels = []
# iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
 # for each document, convert to bag-of-words representation
 bow = dictionary.doc2bow(doc)
 # get list of topic probabilities
 topics = lda_model.get_document_topics(bow)
 # determine topic with highest probability
 dominant_topic = max(topics, key=lambda x: x[1])[0]
 # append to the list
 article_labels.append(dominant_topic)
# Create DataFrame
df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})
# Print the DataFrame
print("Table with Articles and Topic:")
print(df_result)
print()


Table with Articles and Topic:
                                             Article  Topic
0  Rafael Nadal Joins Roger Federer in Missing U....      1
1         Rafael Nadal Is out of the australian open      1
2                     Biden Announces Virus Measures      0
3                   Biden's Virus plans meet reality      0
4                    where biden's virus plan stands      0



In [17]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()


Top terms for Topic #0:
['virus', 'biden', 'plan', 'stand', 'measure', 'announces', 'meet', 'reality', 'australian', 'nadal']

Top terms for Topic #1:
['open', 'rafael', 'nadal', 'federer', 'missing', 'roger', 'join', 'australian', 'biden', 'plan']



### Example 2


In [18]:
df = pd.read_csv('npr.csv')
documents = df['Article'].tolist()

In [20]:
# Create a set of English stopwords
stop_words = set(stopwords.words('english'))

# Initialize a WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize the text into words and convert to lowercase
    tokens = word_tokenize(text.lower())
    
    # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token.isalnum()]
    
    # Remove stopwords from the tokens
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize each token
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Return the preprocessed tokens
    return tokens

# Apply preprocessing to a list of documents
preprocessed_documents = [preprocess_text(doc) for doc in documents]

In [21]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents) 
# Filter out tokens that appear in less than 15 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=15, no_above=0.5)
# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [23]:
# Run LDA
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15) # Train an LDA model 


In [24]:
# empty list to store dominant topic labels for each document
article_labels = []
# iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
 # for each document, convert to bag-of-words representation
 bow = dictionary.doc2bow(doc)
 # get list of topic probabilities
 topics = lda_model.get_document_topics(bow)
 # determine topic with highest probability
 dominant_topic = max(topics, key=lambda x: x[1])[0]
 # append to the list
 article_labels.append(dominant_topic)
# Create DataFrame
df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})
# Print the DataFrame
print("Table with Articles and Topic:")
print(df_result)
print()


Table with Articles and Topic:
                                                 Article  Topic
0      In the Washington of 2016, even when the polic...      1
1        Donald Trump has used Twitter  —   his prefe...      1
2        Donald Trump is unabashedly praising Russian...      1
3      Updated at 2:50 p. m. ET, Russian President Vl...      1
4      From photography, illustration and video, to d...      2
...                                                  ...    ...
11987  The number of law enforcement officers shot an...      0
11988    Trump is busy these days with victory tours,...      1
11989  It’s always interesting for the Goats and Soda...      3
11990  The election of Donald Trump was a surprise to...      1
11991  Voters in the English city of Sunderland did s...      3

[11992 rows x 2 columns]



In [25]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
 print(f"Top terms for Topic #{topic_id}:")
 top_terms = lda_model.show_topic(topic_id, topn=10)
 print([term[0] for term in top_terms])
 print()

Top terms for Topic #0:
['police', 'law', 'court', 'state', 'report', 'case', 'told', 'officer', 'right', 'department']

Top terms for Topic #1:
['trump', 'clinton', 'president', 'republican', 'state', 'campaign', 'election', 'obama', 'vote', 'house']

Top terms for Topic #2:
['know', 'think', 'thing', 'life', 'really', 'woman', 'story', 'show', 'book', 'u']

Top terms for Topic #3:
['food', 'country', 'water', 'world', 'city', 'day', 'china', 'war', '000', 'area']

Top terms for Topic #4:
['health', 'percent', 'school', 'study', 'care', 'student', 'state', 'child', 'company', 'program']



In [27]:
# Print the top terms for each topic with weight
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic:
Topic 0:
- "police" (weight: 0.009)
- "law" (weight: 0.007)
- "court" (weight: 0.007)
- "state" (weight: 0.007)
- "report" (weight: 0.006)
- "case" (weight: 0.006)
- "told" (weight: 0.005)
- "officer" (weight: 0.004)
- "right" (weight: 0.004)
- "department" (weight: 0.004)

Topic 1:
- "trump" (weight: 0.031)
- "clinton" (weight: 0.012)
- "president" (weight: 0.012)
- "republican" (weight: 0.008)
- "state" (weight: 0.008)
- "campaign" (weight: 0.008)
- "election" (weight: 0.007)
- "obama" (weight: 0.006)
- "vote" (weight: 0.006)
- "house" (weight: 0.005)

Topic 2:
- "know" (weight: 0.005)
- "think" (weight: 0.005)
- "thing" (weight: 0.005)
- "life" (weight: 0.005)
- "really" (weight: 0.004)
- "woman" (weight: 0.004)
- "story" (weight: 0.004)
- "show" (weight: 0.003)
- "book" (weight: 0.003)
- "u" (weight: 0.003)

Topic 3:
- "food" (weight: 0.006)
- "country" (weight: 0.005)
- "water" (weight: 0.005)
- "world" (weight: 0.004)
- "city" (weight: 0.004)
- "day" (we